In [1]:
import os
import pandas as pd

# Folder path
folder_path = '/kaggle/input/amazon-products-dataset'

# List to hold all dataframes
dfs = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        df = df.dropna()  # Drop rows with any NaN values
        dfs.append(df)

# Merge all dataframes using 'outer' join
merged_df = pd.concat(dfs, axis=0, join='outer')

# Reset index if necessary
merged_df.reset_index(drop=True, inplace=True)

# Display or save the merged dataframe
merged_df.head()


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0
0,Cave Maruti Suzuki Male-Female Stereo Coupler ...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/31npb8UF2y...,https://www.amazon.in/Maruti-Suzuki-Stereo-Cou...,4.1,7,₹582,₹873,NaN
1,Die Hard Dual Purpose 360-Degree Rotating Mobi...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/61lS3DsWxU...,https://www.amazon.in/Die-Hard-360-Degree-Wind...,5.0,2,₹335,"₹1,299",NaN
2,WEON® Tiny Subwoofer Preamp kit with Volume Co...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/71xLX6DuYK...,https://www.amazon.in/Subwoofer-Preamp-Control...,3.1,4,₹313,₹396,NaN
3,"jiaruixin 6 Volt Kids Ride On Car Charger, 6V ...",car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/61CoaaZnhs...,https://www.amazon.in/Universal-Mercedes-Benz-...,4.2,131,₹593,₹999,NaN
4,Ceuta Retails Motorcycle Phone Holder with USB...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/41p7LZlyhN...,https://www.amazon.in/Ceuta-Retails-Degree-Rot...,3.1,389,₹440,"₹1,599",NaN


In [2]:
merged_df.shape

(681360, 10)

In [3]:
merged_df.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
merged_df['sub_category'].unique()

array(['Car Electronics', 'Janitorial & Sanitation Supplies',
       'All Electronics', 'Make-up', 'Travel Accessories',
       'Car & Bike Care', 'Sunglasses', 'Bags & Luggage', 'Yoga',
       'Sportswear', 'Home Storage', 'Toys Gifting Store',
       'Air Conditioners', 'All Appliances',
       'All Car & Motorbike Products', 'All Exercise & Fitness',
       'All Grocery & Gourmet Foods', 'All Home & Kitchen',
       'All Pet Supplies', 'All Sports, Fitness & Outdoors',
       'Amazon Fashion', 'Baby Bath, Skin & Grooming', 'Baby Fashion',
       'Baby Products', 'Backpacks', 'Badminton', 'Ballerinas',
       'Beauty & Grooming', 'Bedroom Linen', 'Camera Accessories',
       'Cameras', 'Camping & Hiking', 'Car Accessories', 'Car Parts',
       'Cardio Equipment', 'Casual Shoes', 'Clothing',
       'Coffee, Tea & Beverages', 'Cricket', 'Cycling', 'Diapers',
       'Diet & Nutrition', 'Dog supplies', 'Ethnic Wear',
       'Fashion & Silver Jewellery', 'Fashion Sales & Deals',
       'F

In [5]:
merged_df['main_category'].unique()

array(['car & motorbike', 'industrial supplies', 'tv, audio & cameras',
       'beauty & health', 'bags & luggage', 'accessories',
       'sports & fitness', 'stores', 'home & kitchen',
       'toys & baby products', 'appliances', 'grocery & gourmet foods',
       'pet supplies', "kids' fashion", "women's shoes", "men's shoes",
       "women's clothing", "men's clothing", 'music',
       'home, kitchen, pets'], dtype=object)

In [6]:
!pip install sentence_transformers
!pip install hnswlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=205169 sha256=9df5c5749f5681b32182437e973dfdb0a59fc6013eb7531a96432aaaf741a562
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib


In [7]:
import hnswlib
import numpy as np
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
#df_car=pd.read_csv('/kaggle/input/amazon-products-dataset/All Car and Motorbike Products.csv')
#df_ele=pd.read_csv('/kaggle/input/amazon-products-dataset/All Electronics.csv')
#df_sport=pd.read_csv('/kaggle/input/amazon-products-dataset/All Sports Fitness and Outdoors.csv')

In [9]:
data=merged_df

In [10]:
data.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,Cave Maruti Suzuki Male-Female Stereo Coupler ...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/31npb8UF2y...,https://www.amazon.in/Maruti-Suzuki-Stereo-Cou...,4.1,7,₹582,₹873
1,Die Hard Dual Purpose 360-Degree Rotating Mobi...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/61lS3DsWxU...,https://www.amazon.in/Die-Hard-360-Degree-Wind...,5.0,2,₹335,"₹1,299"
2,WEON® Tiny Subwoofer Preamp kit with Volume Co...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/71xLX6DuYK...,https://www.amazon.in/Subwoofer-Preamp-Control...,3.1,4,₹313,₹396
3,"jiaruixin 6 Volt Kids Ride On Car Charger, 6V ...",car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/61CoaaZnhs...,https://www.amazon.in/Universal-Mercedes-Benz-...,4.2,131,₹593,₹999
4,Ceuta Retails Motorcycle Phone Holder with USB...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/41p7LZlyhN...,https://www.amazon.in/Ceuta-Retails-Degree-Rot...,3.1,389,₹440,"₹1,599"


In [11]:
data['main_category'].unique()

array(['car & motorbike', 'industrial supplies', 'tv, audio & cameras',
       'beauty & health', 'bags & luggage', 'accessories',
       'sports & fitness', 'stores', 'home & kitchen',
       'toys & baby products', 'appliances', 'grocery & gourmet foods',
       'pet supplies', "kids' fashion", "women's shoes", "men's shoes",
       "women's clothing", "men's clothing", 'music',
       'home, kitchen, pets'], dtype=object)

In [12]:
dim

NameError: name 'dim' is not defined

# model

In [ ]:
# Step 1: Combine relevant textual features into a single feature for each product
data['combined_features'] = data['name'] + ' ' + data['main_category'] +' '+data['sub_category']

# Step 2: Generate embeddings using a pre-trained Sentence Transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(data['combined_features'].tolist())  # Convert text into embeddings

# Step 3: Initialize HNSW index (if not already initialized)
dim = embeddings.shape[1]  # Dimension of embeddings
num_elements = len(embeddings)

In [ ]:
# Create an HNSW index
hnsw_index = hnswlib.Index(space='l2', dim=dim)  # Using L2 (Euclidean) distance, or 'cosine' for cosine similarity
hnsw_index.init_index(max_elements=num_elements, ef_construction=200, M=16)

# Step 4: Add embeddings to the HNSW index (if not already done)
hnsw_index.add_items(embeddings)

In [ ]:
dim

In [ ]:
def recommend_new_product_similarities(new_product_description, k=10):
    # Generate embedding for the new product (not in the dataset)
    new_product_embedding = model.encode([new_product_description])[0]  # Create embedding for the new product
    
    # Search for the k nearest neighbors using HNSW
    labels, distances = hnsw_index.knn_query(new_product_embedding, k=k)  # Find top k similar products
    
    # Return the similar products from the dataset
    similar_indices = labels.flatten()
    return data.iloc[similar_indices]

In [ ]:
new_product_desc = "car door post"
recommendations = recommend_new_product_similarities(new_product_desc)
recommendations

In [ ]:
location=449384
data['name'].iloc[location]

In [ ]:
data['image'].iloc[location]

# Save data as csv

In [ ]:
data.to_csv('merged_amazon_products.csv', index=False)

# save model files

In [ ]:
# save hnsw
hnsw_index.save_index('hnsw_index_file.hnsw')

In [ ]:
#

to load model
# Create a new index object (with the same parameters as used for training)
hnsw_index = hnswlib.Index(space='l2', dim=128)  # Example parameters: 'l2' for Euclidean, dimension=128

# Load the saved index
hnsw_index.load_index('hnsw_index_file.hnsw')

In [ ]:
import shutil

In [ ]:
model.save('sentence_transformer_model')

In [ ]:
shutil.make_archive('sentence_transformer_model', 'zip', 'sentence_transformer_model')